In [1]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
from datetime import datetime
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import utils as util

In [2]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [3]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, StreetSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2022-01-01T00:00:00.000Z',
        'Maximum': '2023-02-27T15:43:16.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [4]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [5]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [6]:
channelInfoData = channelResponse.json()
data = response.json()

In [7]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

# Put it into function 
We wrap the data request into a nice reusable function and store it under the src folder and cann simply call it in the next step.

In [13]:
# get all data from camera one
camera_one_data = util.return_data(site = "CameraOne", scene = "HospitalView", all_coordinates = True)

In [ ]:
# get all datasets from camera two
camera_two_data_1 = util.return_data(site = "CameraTwo", scene = "SmallFieldSlide", all_coordinates = True)
camera_two_data_2 = util.return_data(site = "CameraTwo", scene = "StreetSlide", all_coordinates = True)
camera_two_data_3 = util.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)

In [ ]:
# merge all four datasets and sort by 
df_all = pd.concat([camera_one_data, camera_two_data_1, camera_two_data_2, camera_two_data_3], 
                   keys=["CameraOne_HospitalView", "CameraTwo_SmallFieldSlide", "CameraTwo_StreetSlide", "CameraTwo_SchoolView"])

In [ ]:
# save data
df_all.to_csv(os.path.abspath(os.path.join('..', 'data'))+'/data_all_cameras.csv', index=False)

In [ ]:
df_all

In [ ]:
# make dataframe completely visible:
from pprint import pprint
from IPython.display import display, HTML

display(HTML(df_all.to_html()))

# To-Do
* allow for coordinate input in query function
* allow to query all different datasets for each site and each scence and combine

In [9]:
# try improved function
import rw_request as rw_rq

In [10]:
camera_two_data_3 = rw_rq.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)